# Google Vertex

In this quickstart you will learn how to run evaluation functions using models from google Vertex.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)]()

In [ ]:
# !pip install trulens trulens-providers-google

In [ ]:
from google import genai

PROJECT_ID = "gemini-experiments-467318"  # 👈 Replace with your real project ID if different
LOCATION = "us-central1"

vertex_client = genai.Client(
    vertexai=True, project=PROJECT_ID, location=LOCATION
)

In [ ]:
response = vertex_client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Tell me about trulens in a few words",
)
print(response.text)

### Create Simple LLM Application

In [ ]:
from trulens.apps.app import instrument
from trulens.core import TruSession


session = TruSession()
session.reset_database()

In [ ]:
class PromptProcessor:
    @instrument
    def generate_completion(self, user_prompt:str) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""Provide a helpful response with relevant background information for the following: {user_prompt}"""

        resp = vertex_client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt,
        )

        return resp.text


simple_application = PromptProcessor()

## Initialize Feedback Function(s)

In [ ]:
from trulens.core import Feedback
from trulens.providers.google import Google

google_vertex_client = Google(vertexai=True, project=PROJECT_ID, location=LOCATION)
relevance = Feedback(google_vertex_client.relevance).on_input_output()
# By default this will check relevance on the main app input and main app
# output.

## Construct the app

Wrap the custom application with `TruApp`, add list of feedbacks for eval

In [ ]:
from trulens.apps.app import TruApp

tru_app = TruApp(
    simple_application,
    app_name="RAG",
    app_version="v1",
    feedbacks=[
        relevance
    ],
)

## Run the app

Use `tru_app` as a context manager for the custom PromptProcessor app.

In [ ]:
with tru_app as recording:
    resp = simple_application.generate_completion("What is a good name for a store that sells colorful socks?")

In [ ]:
print(resp)

## Explore in a Dashboard

In [ ]:
from trulens.dashboard import run_dashboard

run_dashboard(session)